In [3]:
import pandas as pd 

code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0] 
# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format) 

code_df

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DRB동일,004840,고무제품 제조업,"고무벨트(V벨트,콘베이어벨트,평벨트),프라스틱제품 제조,판매",1976-05-21,12월,류영식,http://drbworld.com,부산광역시
1,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
2,GS글로벌,001250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시
3,HDC현대산업개발,294870,건물 건설업,"외주주택, 자체공사, 일반건축, 토목 등",2018-06-12,12월,"권순호, 정경구",http://www.hdc-dvp.com,서울특별시
4,KEC,092220,반도체 제조업,반도체 제조,2006-10-16,12월,박남규 김학남,http://www.kec.co.kr,서울특별시
...,...,...,...,...,...,...,...,...,...
2446,타임기술,318660,소프트웨어 개발 및 공급업,"ILS(종합군수지원), IETM(전자식기술교범), CBT(전자식교보재), 기술번역",2021-06-24,12월,주양효,http://timett.co.kr,경상남도
2447,툴젠,199800,자연과학 및 공학 연구개발업,"유전자가위 (유전체 교정 도구) 및 이를 이용한 유전자세포치료제, 고부가가치 가축,...",2014-06-25,12월,"김영호, 이병화(각자 대표이사)",http://www.toolgen.com,서울특별시
2448,플럼라인생명과학,222670,자연과학 및 공학 연구개발업,"강아지암치료제,돼지DNA테라피",2015-07-28,12월,김앤토니경태,http://www.plsdna.com,서울특별시
2449,한국미라클피플사,331660,기타 화학제품 제조업,"은나노스텝, 닥터오렌지 등",2019-10-28,12월,이호경,http://www.kmpc.co.kr,경기도


In [4]:
# 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다. 
code_df = code_df[['회사명', '종목코드','업종']] 

# 한글로된 컬럼명을 영어로 바꿔준다. 
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code','업종':'category'}) 
code_df.head()



,name,code,category
0,DRB동일,004840,고무제품 제조업
1,DSR,155660,1차 비철금속 제조업
2,GS글로벌,001250,상품 종합 도매업
3,HDC현대산업개발,294870,건물 건설업
4,KEC,092220,반도체 제조업


In [8]:
code_df.query('name=="삼성전자"')['code'].to_string(index=False)

'005930'

In [53]:
# 종목 이름을 입력하면 종목에 해당하는 코드를 불러와 
# 네이버 금융(http://finance.naver.com)에 넣어줌 

def get_url(item_name, code_df): 
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False) 
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code) 
    
    print("요청 URL = {}".format(url)) 
    return url 


# 삼성전자의 일자데이터 url 가져오기 
item_name='NAVER' 
url = get_url(item_name, code_df) 

# 일자 데이터를 담을 df라는 DataFrame 정의 
df = pd.DataFrame() 

import requests


# BeautifulSoup 을 통해 html 페이지 내의 table 태그를 찾습니다.
from bs4 import BeautifulSoup as bs



# html에서 찾은 table 태그를 pandas 로 읽어옵니다.



# 1페이지에서 20페이지의 데이터만 가져오기 
for page in range(1, 201): 
    pg_url = '{url}&page={page}'.format(url=url, page=page) 
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}
    response = requests.get(pg_url, headers=headers)
    html = bs(response.text, "lxml")
    html_table = html.select("table")
    table = pd.read_html(str(html_table))
    table[0].dropna()
    df = df.append(table[0]) 
    
# df.dropna()를 이용해 결측값 있는 행 제거 
df = df.dropna() 

# 상위 5개 데이터 확인하기 
df.head()


요청 URL = http://finance.naver.com/item/sise_day.nhn?code=035420


,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.06.28,408000.0,2000.0,406000.0,413000.0,403000.0,618924.0
2,2021.06.25,410000.0,9500.0,420000.0,420000.0,405000.0,1382051.0
3,2021.06.24,419500.0,4000.0,430000.0,436000.0,413000.0,2143286.0
4,2021.06.23,423500.0,32500.0,393000.0,427000.0,392500.0,2721355.0
5,2021.06.22,391000.0,6000.0,400000.0,400500.0,391000.0,702840.0


In [54]:
# 한글로 된 컬럼명을 영어로 바꿔줌 
df = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'})

# 데이터의 타입을 int형으로 바꿔줌 
df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int)

# 컬럼명 'date'의 타입을 date로 바꿔줌 
df['date'] = pd.to_datetime(df['date']) 

# 일자(date)를 기준으로 오름차순 정렬 
df = df.sort_values(by=['date'], ascending=True)

df.head()


,date,close,diff,open,high,low,volume
13,2013-05-09,288500,12000,298500,301000,286000,436377
12,2013-05-10,290500,2000,289000,295500,289000,256125
11,2013-05-13,292000,1500,290500,293000,281500,302757
10,2013-05-14,289000,3000,286000,291500,285000,246085
9,2013-05-15,293000,4000,288000,296000,286000,399662


In [55]:
!pip install plotly

# 필요한 모듈 import 하기 
import plotly.offline as offline 
import plotly.graph_objs as go 

# jupyter notebook 에서 출력 
offline.init_notebook_mode(connected=True)

trace = go.Scatter( x=df.date, y=df.close, name=item_name)

data = [trace] 

# data = [celltrion] 
layout = dict( 
    title='{}의 종가(close) Time Series'.format(item_name),xaxis=dict( rangeselector=dict( buttons=list([ dict(count=1, label='1m', step='month', stepmode='backward'), dict(count=3, label='3m', step='month', stepmode='backward'), dict(count=6, label='6m', step='month', stepmode='backward'), dict(step='all') ]) ), rangeslider=dict(), type='date' ) )

    
fig = go.Figure(data=data, layout=layout) 
offline.iplot(fig)




In [35]:
df

,date,close,diff,open,high,low,volume
13,2013-05-09,1515000,27000,1490000,1520000,1489000,216123
12,2013-05-10,1476000,39000,1514000,1514000,1476000,235897
11,2013-05-13,1477000,1000,1471000,1480000,1462000,175002
10,2013-05-14,1500000,23000,1487000,1500000,1483000,180053
9,2013-05-15,1500000,0,1485000,1500000,1484000,211896
...,...,...,...,...,...,...,...
5,2021-06-22,80000,100,80200,80300,79900,11773365
4,2021-06-23,80100,100,80500,80600,79900,13856548
3,2021-06-24,81200,1100,80400,81400,80100,18771080
2,2021-06-25,81600,400,81500,81900,81200,13481405


In [44]:
tt = pd.read_csv('./samsung10.csv', encoding='cp949')
tt

,일자,시가,고가,저가,종가,5,10,20,60,120,거래량,5.1,20.1,40,60.1
0,20110325,"17,840","18,200","17,800","18,200",NaN,NaN,NaN,NaN,NaN,"31,115,550",NaN,NaN,NaN,NaN
1,20110328,"18,300","18,440","18,180","18,320",NaN,NaN,NaN,NaN,NaN,"15,910,750",NaN,NaN,NaN,NaN
2,20110329,"18,140","18,320","18,000","18,320",NaN,NaN,NaN,NaN,NaN,"17,231,200",NaN,NaN,NaN,NaN
3,20110330,"18,320","18,580","18,320","18,520",NaN,NaN,NaN,NaN,NaN,"20,712,850",NaN,NaN,NaN,NaN
4,20110331,"18,720","18,720","18,420","18,640","18,400",NaN,NaN,NaN,NaN,"26,383,300","22,270,730",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2515,20210615,"80,900","81,200","80,600","80,900","80,900","81,410","80,675","81,937","82,539","10,075,685","13,441,909","15,875,412","17,342,923","16,943,059"
2516,20210616,"81,500","81,900","81,100","81,800","81,040","81,510","80,785","81,933","82,613","14,999,855","13,460,115","15,750,684","17,335,737","16,981,881"
2517,20210617,"81,100","81,300","80,700","80,900","81,020","81,320","80,855","81,918","82,678","14,007,385","12,744,193","15,623,962","17,297,873","16,993,673"
2518,20210618,"81,100","81,100","80,500","80,500","80,920","81,150","80,875","81,910","82,747","14,916,721","12,909,945","15,321,712","17,129,889","16,943,507"


In [48]:
tt[tt['일자']==20180503]


,일자,시가,고가,저가,종가,5,10,20,60,120,거래량,5.1,20.1,40,60.1
1753,20180503,"53,000","53,000","53,000","53,000","52,828","52,130","50,793","49,263","50,435",0,"9,671,470","11,496,678","11,552,490","13,100,534"
